In [1]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = (10, 8)
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import collections
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
# рори тащит и доставляет https://coub.com/view/axsih
# историю мучений с пачкой memory error опустим, она ужаснаа  хоть плач.
# отсутствие сильного справочного суппорта доcтавляет, никто из DS не любит писать функции.
# ..не любит потом что время тестирования одной несчастной фукнции в 3 строчки на большой выборке - околокосмическое.

In [4]:
df_1 = pd.read_csv('J:/Data_mks/sub_rebills_2017-08-01_2017-08-31_f/sub_rebills_2017-08-01_2017-08-31_f.csv', sep='\t', encoding='utf-8')

In [5]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000730 entries, 0 to 5000729
Data columns (total 21 columns):
sub_uuid             object
service_id           int64
URL                  object
Подключение          object
Юр.лицо              object
ОСС                  object
date_init            object
date_activation      object
date_stop            object
msisdn               int64
referrer             object
query                object
ip                   object
ua                   object
error_code           object
is_active            object
rebill_uuid          object
rebill_date          object
rebill_error_code    float64
rebill_status        float64
ua_black_list        int64
dtypes: float64(2), int64(3), object(16)
memory usage: 801.2+ MB


In [6]:
df_1.head()

,sub_uuid,service_id,URL,Подключение,Юр.лицо,ОСС,date_init,date_activation,date_stop,msisdn,...,query,ip,ua,error_code,is_active,rebill_uuid,rebill_date,rebill_error_code,rebill_status,ua_black_list
0,3c9710c0-7633-11e7-a779-718ddddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-01 00:00:09,NaN,NaN,0,...,utm_source=das&tmpl=amedia_got&exchange=axonix...,85.140.4.140,Mozilla/5.0 (Linux; Android 6.0; U20 Build/MRA...,0,0,NaN,NaN,NaN,NaN,0
1,3dd8f1b0-7633-11e7-aa86-6e8ddddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-01 00:00:11,NaN,NaN,0,...,utm_source=das&tmpl=amedia_def&exchange=axonix...,85.140.1.117,Mozilla/5.0 (Linux; Android 4.4.2; ASUS_T00J B...,0,0,NaN,NaN,NaN,NaN,0
2,3e7bb936-7633-11e7-b3ea-728ddddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-01 00:00:12,NaN,NaN,0,...,utm_source=das&tmpl=amedia_def&exchange=epom&f...,213.87.163.0,Mozilla/5.0 (Linux; Android 6.0; MZ-MEIZU_M5 B...,0,0,NaN,NaN,NaN,NaN,1
3,44c4eeb6-7633-11e7-91ac-738ddddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-01 00:00:22,NaN,NaN,0,...,utm_source=das&tmpl=amedia_got2&exchange=epom&...,213.87.152.167,Mozilla/5.0 (Linux; Android 5.0.1; Lenovo TAB ...,0,0,NaN,NaN,NaN,NaN,0
4,45530142-7633-11e7-a779-718ddddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-01 00:00:23,NaN,NaN,0,...,utm_source=das&tmpl=amedia_got&exchange=epom&f...,213.87.128.9,Mozilla/5.0 (Linux; Android 4.3; GT-I9500 Buil...,0,0,NaN,NaN,NaN,NaN,0


In [29]:
# global в топку - генератор Варп_полей к бою, поня Хаоса решила погулять по Варп лужайкам через Варп порталы.

def select():
    Bexc = df_1[df_1['query'].str.contains('exchange=%7Bexc%7D', na=False)]
    BEXCHANGE_ID = df_1[df_1['query'].str.contains('exchange=BEXCHANGE_ID', na=False)]
    EXCHANGE_ID = df_1[df_1['query'].str.contains('exchange=EXCHANGE_ID', na=False)]
    axonix = df_1[df_1['query'].str.contains('exchange=axonix', na=False)]
    smartyads = df_1[df_1['query'].str.contains('exchange=smartyads', na=False)]
    undef = df_1[df_1['query'].str.contains('exchange=undef', na=False)]
    arteebee = df_1[df_1['query'].str.contains('exchange=arteebee', na=False)]
    between = df_1[df_1['query'].str.contains('exchange=between', na=False)]
    clickky = df_1[df_1['query'].str.contains('exchange=clickky', na=False)]
    google = df_1[df_1['query'].str.contains('exchange=google', na=False)]
    gothamads = df_1[df_1['query'].str.contains('exchange=gothamads', na=False)]
    mobilda = df_1[df_1['query'].str.contains('exchange=mobilda', na=False)]
    repeater = df_1[df_1['query'].str.contains('exchange=repeater', na=False)]
    republer = df_1[df_1['query'].str.contains('exchange=republer', na=False)]
    sape = df_1[df_1['query'].str.contains('exchange=sape', na=False)]  
    clickky2 = df_1[df_1['query'].str.contains('exchange=clickky2', na=False)]
    exoclick = df_1[df_1['query'].str.contains('exchange=exoclick', na=False)]
    popcash = df_1[df_1['query'].str.contains('exchange=popcash', na=False)]
    popunder = df_1[df_1['query'].str.contains('exchange=popunder', na=False)]
    propeller = df_1[df_1['query'].str.contains('exchange=propeller', na=False)]
    yabbi = df_1[df_1['query'].str.contains('exchange=yabbi', na=False)]
    activerevenue = df_1[df_1['query'].str.contains('exchange=activerevenue', na=False)]
    fallback = df_1[df_1['query'].str.contains('exchange=fallback', na=False)]
    innex = df_1[df_1['query'].str.contains('exchange=innex', na=False)]
    juicy = df_1[df_1['query'].str.contains('exchange=juicy', na=False)]
    airpush = df_1[df_1['query'].str.contains('exchange=airpush', na=False)]
    leadbolt = df_1[df_1['query'].str.contains('exchange=leadbolt', na=False)]
    something = df_1[df_1['query'].str.contains('exchange=something', na=False)]
    hilltop = df_1[df_1['query'].str.contains('exchange=hilltop', na=False)]
    propellerads = df_1[df_1['query'].str.contains('exchange=propellerads', na=False)]
    mobicow = df_1[df_1['query'].str.contains('exchange=mobicow', na=False)]
    openx = df_1[df_1['query'].str.contains('exchange=openx', na=False)]
    smaato = df_1[df_1['query'].str.contains('exchange=smaato', na=False)]
    tonic = df_1[df_1['query'].str.contains('exchange=tonic', na=False)]
    traffichunt = df_1[df_1['query'].str.contains('exchange=traffichunt', na=False)]
    zeropark = df_1[df_1['query'].str.contains('exchange=zeropark', na=False)]
    admobile = df_1[df_1['query'].str.contains('exchange=admobile', na=False)]
    tapxx = df_1[df_1['query'].str.contains('exchange=tapxx', na=False)]
    teasernet = df_1[df_1['query'].str.contains('exchange=teasernet', na=False)]
    visitweb = df_1[df_1['query'].str.contains('exchange=visitweb', na=False)]
    epom = df_1[df_1['query'].str.contains('exchange=epom', na=False)]
    plugrush = df_1[df_1['query'].str.contains('exchange=plugrush', na=False)]
    kadam = df_1[df_1['query'].str.contains('exchange=kadam', na=False)]
    minimob = df_1[df_1['query'].str.contains('exchange=minimob', na=False)]
    peakadx = df_1[df_1['query'].str.contains('exchange=peakadx', na=False)]
    a_propellerads = df_1[df_1['query'].str.contains('exchange=propellerads', na=False)]
    print('Bexc', len(Bexc), 'axonix', len(axonix), 'smartyads', len(smartyads), 'undef', len(undef),'arteebee', len(arteebee), 
                 'between', len(between), 'clickky', len(clickky), 'google', len(google), 'gothamads',len(gothamads), 'mobilda',len(mobilda),
                 'repeater',len(repeater), 'republer', len(republer), 'sape', len(sape), 'clickky2', len(clickky2),'exoclick',len(exoclick),
                 'popcash',len(popcash), 'popunder',len(popunder), 'propeller',len(propeller), 'yabbi',len(yabbi), 'activerevenue', len(activerevenue),
                'fallback', len(fallback), 'innex', len(innex), 'juicy',len(juicy), 'airpush', len(airpush), 'leadbolt', len(leadbolt),
                 'something', len(something), 'hilltop',len(hilltop), 'propellerads',len(propellerads), 'mobicow', len(mobicow),'openx',len(openx),
                 'smaato',len(smaato), 'tonic',len(tonic), 'traffichunt',len(traffichunt), 'zeropark', len(zeropark), 'admobile', len(admobile),
                 'tapxx', len(tapxx), 'teasernet', len(teasernet), 'visitweb', len(visitweb), 'BEXCHANGE_ID',len(BEXCHANGE_ID),
                 'EXCHANGE_ID',len(EXCHANGE_ID) ,'epom', len(epom), 'plugrush',len(plugrush), 'kadam', len(kadam),
                  'minimob', len(minimob), 'peakadx', len(peakadx), 'a_propellerads', len(a_propellerads))
    return Bexc,BEXCHANGE_ID,EXCHANGE_ID,axonix,smartyads,undef,arteebee,between,clickky,google,gothamads,mobilda,repeater,republer,sape,clickky2,exoclick, popcash,popunder,propeller,yabbi,activerevenue,fallback,innex,juicy,airpush, leadbolt,
    something, hilltop,propellerads,mobicow,openx, smaato, tonic,traffichunt,zeropark,admobile,tapxx,teasernet,visitweb, epom,
    plugrush,kadam, minimob,peakadx,a_propellerads
        
    
heresy_list = select()

def warp_field_generator():
    for pony in heresy_list:
        yield pony

warp_portal = warp_field_generator()

Bexc 2 axonix 1263244 smartyads 36737 undef 0 arteebee 1 between 2902 clickky 1138501 google 745 gothamads 3705 mobilda 588196 repeater 1774 republer 3 sape 223286 clickky2 0 exoclick 0 popcash 0 popunder 0 propeller 0 yabbi 0 activerevenue 0 fallback 0 innex 0 juicy 0 airpush 0 leadbolt 0 something 0 hilltop 0 propellerads 0 mobicow 0 openx 0 smaato 0 tonic 0 traffichunt 0 zeropark 0 admobile 0 tapxx 0 teasernet 0 visitweb 0 BEXCHANGE_ID 0 EXCHANGE_ID 0 epom 576375 plugrush 0 kadam 0 minimob 0 peakadx 0 a_propellerads 0


In [30]:
next(warp_portal)

,sub_uuid,service_id,URL,Подключение,Юр.лицо,ОСС,date_init,date_activation,date_stop,msisdn,...,query,ip,ua,error_code,is_active,rebill_uuid,rebill_date,rebill_error_code,rebill_status,ua_black_list
638986,3604aa2c-7b9e-11e7-953f-bf57dddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-07 21:28:30,NaN,NaN,0,...,utm_source=das&tmpl=amedia_def&utm_source=g2m&...,75.156.75.93,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,8,0,NaN,NaN,NaN,NaN,0
638990,36409af0-7b9e-11e7-aa3e-c157dddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-07 21:28:30,NaN,NaN,0,...,utm_source=das&tmpl=amedia_def&utm_source=g2m&...,75.156.75.93,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,8,0,NaN,NaN,NaN,NaN,0


In [31]:
next(warp_portal)

,sub_uuid,service_id,URL,Подключение,Юр.лицо,ОСС,date_init,date_activation,date_stop,msisdn,...,query,ip,ua,error_code,is_active,rebill_uuid,rebill_date,rebill_error_code,rebill_status,ua_black_list


In [14]:
next(warp_portal)

,sub_uuid,service_id,URL,Подключение,Юр.лицо,ОСС,date_init,date_activation,date_stop,msisdn,...,query,ip,ua,error_code,is_active,rebill_uuid,rebill_date,rebill_error_code,rebill_status,ua_black_list


In [15]:
next(warp_portal)

,sub_uuid,service_id,URL,Подключение,Юр.лицо,ОСС,date_init,date_activation,date_stop,msisdn,...,query,ip,ua,error_code,is_active,rebill_uuid,rebill_date,rebill_error_code,rebill_status,ua_black_list
0,3c9710c0-7633-11e7-a779-718ddddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-01 00:00:09,NaN,NaN,0,...,utm_source=das&tmpl=amedia_got&exchange=axonix...,85.140.4.140,Mozilla/5.0 (Linux; Android 6.0; U20 Build/MRA...,0,0,NaN,NaN,NaN,NaN,0
1,3dd8f1b0-7633-11e7-aa86-6e8ddddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-01 00:00:11,NaN,NaN,0,...,utm_source=das&tmpl=amedia_def&exchange=axonix...,85.140.1.117,Mozilla/5.0 (Linux; Android 4.4.2; ASUS_T00J B...,0,0,NaN,NaN,NaN,NaN,0
12,54234664-7633-11e7-aa86-6e8ddddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-01 00:00:48,NaN,NaN,0,...,utm_source=das&tmpl=amedia_def&exchange=axonix...,85.140.79.95,Mozilla/5.0 (Linux; Android 5.1; HUAWEI CUN-U2...,0,0,NaN,NaN,NaN,NaN,0
16,64b47714-7633-11e7-a779-718ddddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-01 00:01:16,NaN,NaN,0,...,utm_source=das&tmpl=amedia_def&exchange=axonix...,85.140.6.79,Mozilla/5.0 (Linux; Android 5.1; Roverphone Bu...,0,0,NaN,NaN,NaN,NaN,1
20,6fad139c-7633-11e7-a779-718ddddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-01 00:01:34,NaN,NaN,0,...,utm_source=das&tmpl=amedia_def&exchange=axonix...,85.140.78.248,Mozilla/5.0 (Linux; Android 5.0.2; SM-A500F Bu...,0,0,NaN,NaN,NaN,NaN,0
23,744030f6-7633-11e7-a779-718ddddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-01 00:01:42,NaN,NaN,0,...,utm_source=das&tmpl=amedia_got&exchange=axonix...,213.87.123.166,Mozilla/5.0 (Linux; Android 6.0; CRO-L22 Build...,0,0,NaN,NaN,NaN,NaN,0
29,7d15c236-7633-11e7-a779-718ddddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-01 00:01:57,NaN,NaN,0,...,utm_source=das&tmpl=amedia_def&exchange=axonix...,85.140.0.11,Mozilla/5.0 (Linux; Android 4.4.2; ZTE Blade L...,0,0,NaN,NaN,NaN,NaN,0
31,7eec6704-7633-11e7-91ac-738ddddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-01 00:02:00,NaN,NaN,0,...,utm_source=das&tmpl=amedia_def&exchange=axonix...,213.87.100.131,Mozilla/5.0 (Linux; Android 4.4.2; GT-I9200 Bu...,8,0,NaN,NaN,NaN,NaN,0
33,7f78f78c-7633-11e7-aa86-6e8ddddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-01 00:02:01,NaN,NaN,0,...,utm_source=das&tmpl=amedia_def&exchange=axonix...,85.140.3.59,Mozilla/5.0 (Linux; Android 6.0.1; MotoG3 Buil...,0,0,NaN,NaN,NaN,NaN,0
36,82388a1e-7633-11e7-91ac-738ddddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-01 00:02:06,NaN,NaN,0,...,utm_source=das&tmpl=amedia_got&exchange=axonix...,85.140.1.117,Mozilla/5.0 (Linux; Android 4.4.2; ASUS_T00J B...,0,0,NaN,NaN,NaN,NaN,0


In [33]:
next(warp_portal)

,sub_uuid,service_id,URL,Подключение,Юр.лицо,ОСС,date_init,date_activation,date_stop,msisdn,...,query,ip,ua,error_code,is_active,rebill_uuid,rebill_date,rebill_error_code,rebill_status,ua_black_list
0,3c9710c0-7633-11e7-a779-718ddddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-01 00:00:09,NaN,NaN,0,...,utm_source=das&tmpl=amedia_got&exchange=axonix...,85.140.4.140,Mozilla/5.0 (Linux; Android 6.0; U20 Build/MRA...,0,0,NaN,NaN,NaN,NaN,0
1,3dd8f1b0-7633-11e7-aa86-6e8ddddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-01 00:00:11,NaN,NaN,0,...,utm_source=das&tmpl=amedia_def&exchange=axonix...,85.140.1.117,Mozilla/5.0 (Linux; Android 4.4.2; ASUS_T00J B...,0,0,NaN,NaN,NaN,NaN,0
12,54234664-7633-11e7-aa86-6e8ddddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-01 00:00:48,NaN,NaN,0,...,utm_source=das&tmpl=amedia_def&exchange=axonix...,85.140.79.95,Mozilla/5.0 (Linux; Android 5.1; HUAWEI CUN-U2...,0,0,NaN,NaN,NaN,NaN,0
16,64b47714-7633-11e7-a779-718ddddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-01 00:01:16,NaN,NaN,0,...,utm_source=das&tmpl=amedia_def&exchange=axonix...,85.140.6.79,Mozilla/5.0 (Linux; Android 5.1; Roverphone Bu...,0,0,NaN,NaN,NaN,NaN,1
20,6fad139c-7633-11e7-a779-718ddddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-01 00:01:34,NaN,NaN,0,...,utm_source=das&tmpl=amedia_def&exchange=axonix...,85.140.78.248,Mozilla/5.0 (Linux; Android 5.0.2; SM-A500F Bu...,0,0,NaN,NaN,NaN,NaN,0
23,744030f6-7633-11e7-a779-718ddddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-01 00:01:42,NaN,NaN,0,...,utm_source=das&tmpl=amedia_got&exchange=axonix...,213.87.123.166,Mozilla/5.0 (Linux; Android 6.0; CRO-L22 Build...,0,0,NaN,NaN,NaN,NaN,0
29,7d15c236-7633-11e7-a779-718ddddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-01 00:01:57,NaN,NaN,0,...,utm_source=das&tmpl=amedia_def&exchange=axonix...,85.140.0.11,Mozilla/5.0 (Linux; Android 4.4.2; ZTE Blade L...,0,0,NaN,NaN,NaN,NaN,0
31,7eec6704-7633-11e7-91ac-738ddddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-01 00:02:00,NaN,NaN,0,...,utm_source=das&tmpl=amedia_def&exchange=axonix...,213.87.100.131,Mozilla/5.0 (Linux; Android 4.4.2; GT-I9200 Bu...,8,0,NaN,NaN,NaN,NaN,0
33,7f78f78c-7633-11e7-aa86-6e8ddddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-01 00:02:01,NaN,NaN,0,...,utm_source=das&tmpl=amedia_def&exchange=axonix...,85.140.3.59,Mozilla/5.0 (Linux; Android 6.0.1; MotoG3 Buil...,0,0,NaN,NaN,NaN,NaN,0
36,82388a1e-7633-11e7-91ac-738ddddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-01 00:02:06,NaN,NaN,0,...,utm_source=das&tmpl=amedia_got&exchange=axonix...,85.140.1.117,Mozilla/5.0 (Linux; Android 4.4.2; ASUS_T00J B...,0,0,NaN,NaN,NaN,NaN,0


In [34]:
next(warp_portal)

,sub_uuid,service_id,URL,Подключение,Юр.лицо,ОСС,date_init,date_activation,date_stop,msisdn,...,query,ip,ua,error_code,is_active,rebill_uuid,rebill_date,rebill_error_code,rebill_status,ua_black_list
1785,bca1d2d0-7644-11e7-b3ea-728ddddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-01 02:05:25,NaN,NaN,0,...,utm_source=das&tmpl=amedia_def&exchange=smarty...,217.28.215.228,Mozilla/5.0 (Linux; Android 6.0.1; SAMSUNG SM-...,8,0,NaN,NaN,NaN,NaN,0
595703,615f78a8-7b55-11e7-86f7-667cdddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-07 12:47:09,NaN,NaN,0,...,utm_source=das&tmpl=amedia_got&exchange=smarty...,95.153.134.49,Mozilla/5.0 (Linux; Android 5.1; LG-X210 Build...,0,0,NaN,NaN,NaN,NaN,0
595723,6b0972dc-7b55-11e7-b543-677cdddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-07 12:47:25,NaN,NaN,0,...,utm_source=das&tmpl=amedia_def&exchange=smarty...,95.153.128.144,Mozilla/5.0 (Linux; Android 5.0.2; SM-A300F Bu...,0,0,NaN,NaN,NaN,NaN,0
595737,74d92a0a-7b55-11e7-86f7-667cdddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-07 12:47:42,NaN,NaN,0,...,utm_source=das&tmpl=amedia_got2&exchange=smart...,85.140.0.178,Mozilla/5.0 (Linux; Android 5.1; 5019D Build/L...,0,0,NaN,NaN,NaN,NaN,1
595882,b1747c30-7b55-11e7-b918-657cdddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-07 12:49:23,NaN,NaN,0,...,utm_source=das&tmpl=amedia_got&exchange=smarty...,85.140.7.21,Mozilla/5.0 (Linux; Android 6.0.1; D6503 Build...,0,0,NaN,NaN,NaN,NaN,0
595886,b1cac4d2-7b55-11e7-aea0-647cdddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-07 12:49:24,NaN,NaN,0,...,utm_source=das&tmpl=amedia_def&exchange=smarty...,217.66.158.54,Mozilla/5.0 (Linux; Android 6.0.1; D6653 Build...,0,0,NaN,NaN,NaN,NaN,0
596135,0487e4fc-7b56-11e7-86f7-667cdddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-07 12:51:43,NaN,NaN,0,...,utm_source=das&tmpl=amedia_got2&exchange=smart...,95.153.131.69,Mozilla/5.0 (Linux; Android 5.1; SMART_Surf_4G...,0,0,NaN,NaN,NaN,NaN,1
596147,081c873a-7b56-11e7-86f7-667cdddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-07 12:51:49,NaN,NaN,0,...,utm_source=das&tmpl=amedia_def&exchange=smarty...,80.83.238.57,Mozilla/5.0 (Linux; Android 4.4.2; GT-I9301I B...,0,0,NaN,NaN,NaN,NaN,0
596191,16525582-7b56-11e7-b918-657cdddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-07 12:52:13,NaN,NaN,0,...,utm_source=das&tmpl=amedia_got&exchange=smarty...,95.153.131.46,Mozilla/5.0 (Linux; Android 4.4.2; GT-I9190 Bu...,0,0,NaN,NaN,NaN,NaN,0
596349,4ec0cf84-7b56-11e7-86f7-667cdddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-07 12:53:47,NaN,NaN,0,...,utm_source=das&tmpl=amedia_def&exchange=smarty...,85.140.7.38,Mozilla/5.0 (Linux; Android 4.4.2; Lenovo A606...,0,0,NaN,NaN,NaN,NaN,0


In [35]:
next(warp_portal)

,sub_uuid,service_id,URL,Подключение,Юр.лицо,ОСС,date_init,date_activation,date_stop,msisdn,...,query,ip,ua,error_code,is_active,rebill_uuid,rebill_date,rebill_error_code,rebill_status,ua_black_list


In [36]:
next(warp_portal)

,sub_uuid,service_id,URL,Подключение,Юр.лицо,ОСС,date_init,date_activation,date_stop,msisdn,...,query,ip,ua,error_code,is_active,rebill_uuid,rebill_date,rebill_error_code,rebill_status,ua_black_list
4999984,2a9b00b6-8e8e-11e7-9e1e-75ff3b4b643c,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-31 23:51:31,NaN,NaN,0,...,utm_source=das&exchange=arteebee,78.107.65.18,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6...,8,0,NaN,NaN,NaN,NaN,0


In [37]:
next(warp_portal)

,sub_uuid,service_id,URL,Подключение,Юр.лицо,ОСС,date_init,date_activation,date_stop,msisdn,...,query,ip,ua,error_code,is_active,rebill_uuid,rebill_date,rebill_error_code,rebill_status,ua_black_list
1889140,f107c67a-8430-11e7-b314-598bdddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-18 19:20:51,NaN,NaN,0,...,utm_source=das&tmpl=amedia_got5&exchange=betwe...,109.106.141.72,Mozilla/5.0 (Linux; Android 5.1.1; SM-G531H Bu...,0,0,NaN,NaN,NaN,NaN,0
1891937,54e2f43e-8432-11e7-b314-598bdddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-18 19:30:34,NaN,NaN,0,...,utm_source=das&tmpl=amedia_got5&exchange=betwe...,46.56.184.6,Mozilla/5.0 (Linux; Android 5.1.1; ZTE T617 Bu...,8,0,NaN,NaN,NaN,NaN,0
1898359,3c8f59ec-8435-11e7-b314-598bdddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-18 19:51:32,NaN,NaN,0,...,utm_source=das&tmpl=amedia_got5&exchange=betwe...,177.234.178.16,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,8,0,NaN,NaN,NaN,NaN,0
1903908,c8b8bb50-8437-11e7-b314-598bdddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-18 20:09:53,NaN,NaN,0,...,utm_source=das&tmpl=amedia_got5&exchange=betwe...,84.47.150.10,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....,8,0,NaN,NaN,NaN,NaN,0
1905765,a8b1a82a-8438-11e7-b314-598bdddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-18 20:15:55,NaN,NaN,0,...,utm_source=das&tmpl=amedia_got5&exchange=betwe...,77.94.198.207,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,8,0,NaN,NaN,NaN,NaN,0
1905873,b0be4fbe-8438-11e7-b314-598bdddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-18 20:16:11,NaN,NaN,0,...,utm_source=das&tmpl=amedia_got5&exchange=betwe...,178.206.28.160,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,8,0,NaN,NaN,NaN,NaN,0
1907077,442304fc-8439-11e7-b314-598bdddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-18 20:19:56,NaN,NaN,0,...,utm_source=das&tmpl=amedia_got5&exchange=betwe...,190.103.220.2,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,8,0,NaN,NaN,NaN,NaN,0
1907541,6e907256-8439-11e7-b314-598bdddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-18 20:21:20,NaN,NaN,0,...,utm_source=das&tmpl=amedia_got5&exchange=betwe...,46.70.120.77,Mozilla/5.0 (Linux; Android 4.4.2; ARK_Benefit...,8,0,NaN,NaN,NaN,NaN,1
1907681,7e2c0fcc-8439-11e7-b314-598bdddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-18 20:21:37,NaN,NaN,0,...,utm_source=das&tmpl=amedia_got5&exchange=betwe...,46.70.120.77,Mozilla/5.0 (Linux; Android 4.4.2; ARK_Benefit...,8,0,NaN,NaN,NaN,NaN,1
1909475,4297c91e-843a-11e7-b314-598bdddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-18 20:27:15,NaN,NaN,0,...,utm_source=das&tmpl=amedia_got5&exchange=betwe...,193.124.64.45,Opera/9.80 (Windows NT 6.1; WOW64) Presto/2.12...,8,0,NaN,NaN,NaN,NaN,0


In [38]:
next(warp_portal)

,sub_uuid,service_id,URL,Подключение,Юр.лицо,ОСС,date_init,date_activation,date_stop,msisdn,...,query,ip,ua,error_code,is_active,rebill_uuid,rebill_date,rebill_error_code,rebill_status,ua_black_list
1165597,b8046e86-80d0-11e7-b314-598bdddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-14 12:13:40,NaN,NaN,0,...,utm_source=das&tmpl=amedia_got2&exchange=click...,85.140.3.232,Mozilla/5.0 (Linux; Android 6.0.1; ASUS_Z00ED ...,0,0,NaN,NaN,NaN,NaN,0
1165647,c3aa22b2-80d0-11e7-b314-598bdddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-14 12:14:07,NaN,NaN,0,...,utm_source=das&tmpl=amedia_got2&exchange=click...,213.87.127.11,Mozilla/5.0 (Linux; Android 6.0; Power Rage Ev...,0,0,NaN,NaN,NaN,NaN,0
1165658,c8bf0af6-80d0-11e7-b314-598bdddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-14 12:14:13,NaN,NaN,0,...,utm_source=das&tmpl=amedia_got&exchange=clickk...,95.153.130.225,Mozilla/5.0 (Linux; Android 5.1; 5019D Build/L...,0,0,NaN,NaN,NaN,NaN,1
1165671,cdf6eab6-80d0-11e7-b314-598bdddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-14 12:14:20,NaN,NaN,0,...,utm_source=das&tmpl=amedia_got&exchange=clickk...,85.140.3.193,Mozilla/5.0 (Linux; Android 5.0.1; GT-I9500 Bu...,0,0,NaN,NaN,NaN,NaN,0
1165685,d2d08f6a-80d0-11e7-b314-598bdddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-14 12:14:28,NaN,NaN,0,...,utm_source=das&tmpl=amedia_got2&exchange=click...,213.87.122.140,Mozilla/5.0 (Linux; Android 6.0; F3311 Build/3...,0,0,NaN,NaN,NaN,NaN,0
1165694,d517d454-80d0-11e7-b314-598bdddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-14 12:14:32,NaN,NaN,0,...,utm_source=das&tmpl=amedia_got2&exchange=click...,213.87.156.170,Mozilla/5.0 (Linux; Android 5.1; Micromax D333...,0,0,NaN,NaN,NaN,NaN,1
1165701,d8d795a2-80d0-11e7-b314-598bdddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-14 12:14:34,NaN,NaN,0,...,utm_source=das&tmpl=amedia_def&exchange=clickk...,85.140.78.80,Mozilla/5.0 (Linux; Android 6.0; UMI_London Bu...,0,0,NaN,NaN,NaN,NaN,1
1165705,daa9c88c-80d0-11e7-b314-598bdddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-14 12:14:36,NaN,NaN,0,...,utm_source=das&tmpl=amedia_got2&exchange=click...,213.87.224.128,Mozilla/5.0 (Linux; Android 6.0.1; SM-G570F Bu...,0,0,NaN,NaN,NaN,NaN,0
1165715,dd45bb3c-80d0-11e7-b314-598bdddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-14 12:14:41,NaN,NaN,0,...,utm_source=das&tmpl=amedia_def&exchange=clickk...,85.140.3.12,Mozilla/5.0 (Linux; Android 5.1; Philips_V526 ...,0,0,NaN,NaN,NaN,NaN,0
1165724,e0036450-80d0-11e7-b314-598bdddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-14 12:14:44,NaN,NaN,0,...,utm_source=das&tmpl=amedia_got&exchange=clickk...,85.140.2.15,Mozilla/5.0 (Linux; Android 6.0.1; Redmi 4X Bu...,0,0,NaN,NaN,NaN,NaN,0


In [39]:
next(warp_portal)

,sub_uuid,service_id,URL,Подключение,Юр.лицо,ОСС,date_init,date_activation,date_stop,msisdn,...,query,ip,ua,error_code,is_active,rebill_uuid,rebill_date,rebill_error_code,rebill_status,ua_black_list
49139,a6770de6-776f-11e7-8831-9c5cdddfe09a,1013,Амедиатека,mks (478),МКС ООО,МТС,2017-08-02 13:45:07,NaN,NaN,0,...,utm_source=das&exchange=google&type=image&test...,66.249.89.34,AdsBot-Google (+http://www.google.com/adsbot.h...,8,0,NaN,NaN,NaN,NaN,0
50353,70a66da6-7774-11e7-a963-d263dddfe09a,1013,Амедиатека,mks (478),МКС ООО,МТС,2017-08-02 14:19:24,NaN,NaN,0,...,utm_source=das&exchange=google&type=image&test...,78.107.65.18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,8,0,NaN,NaN,NaN,NaN,0
50439,c1fef286-7774-11e7-a963-d263dddfe09a,1013,Амедиатека,mks (478),МКС ООО,МТС,2017-08-02 14:21:41,NaN,NaN,0,...,utm_source=das&exchange=google&type=image&test...,78.107.65.18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,8,0,NaN,NaN,NaN,NaN,0
50450,cc14b4ea-7774-11e7-b077-d463dddfe09a,1013,Амедиатека,mks (478),МКС ООО,МТС,2017-08-02 14:21:58,NaN,NaN,0,...,utm_source=das&exchange=google&type=image&test...,78.107.65.18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,8,0,NaN,NaN,NaN,NaN,0
50738,3ee3a35e-7776-11e7-b077-d463dddfe09a,1013,Амедиатека,mks (478),МКС ООО,МТС,2017-08-02 14:32:20,NaN,NaN,0,...,utm_source=das&exchange=google&type=image&test...,78.107.65.18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,8,0,NaN,NaN,NaN,NaN,0
50853,aff32f1a-7776-11e7-8087-d563dddfe09a,1013,Амедиатека,mks (478),МКС ООО,МТС,2017-08-02 14:35:30,NaN,NaN,0,...,utm_source=das&exchange=google&type=image&test...,104.132.20.87,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,8,0,NaN,NaN,NaN,NaN,0
50869,c0234c94-7776-11e7-8087-d563dddfe09a,1013,Амедиатека,mks (478),МКС ООО,МТС,2017-08-02 14:35:57,NaN,NaN,0,...,utm_source=das&exchange=google&type=image&test...,104.132.20.87,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,8,0,NaN,NaN,NaN,NaN,0
52491,ffd98654-777c-11e7-8087-d563dddfe09a,1013,Амедиатека,mks (478),МКС ООО,МТС,2017-08-02 15:20:41,NaN,NaN,0,...,utm_source=das&exchange=google&type=image&test...,104.132.20.87,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,8,0,NaN,NaN,NaN,NaN,0
52803,fc8324e6-777d-11e7-8506-d363dddfe09a,1013,Амедиатека,mks (478),МКС ООО,МТС,2017-08-02 15:27:45,NaN,NaN,0,...,utm_source=das&exchange=google&type=image&test...,104.132.20.87,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,8,0,NaN,NaN,NaN,NaN,0
53005,9d29bab8-777e-11e7-8087-d563dddfe09a,1013,Амедиатека,mks (478),МКС ООО,МТС,2017-08-02 15:32:15,NaN,NaN,0,...,utm_source=das&exchange=google&type=image&test...,104.132.20.87,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,8,0,NaN,NaN,NaN,NaN,0


In [40]:
next(warp_portal)

,sub_uuid,service_id,URL,Подключение,Юр.лицо,ОСС,date_init,date_activation,date_stop,msisdn,...,query,ip,ua,error_code,is_active,rebill_uuid,rebill_date,rebill_error_code,rebill_status,ua_black_list
3721731,a8e8e68f-0c6d-4982-b142-899b55966239,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-25 10:26:05,NaN,NaN,0,...,uuid=a8e8e68f-0c6d-4982-b142-899b55966239&serv...,95.153.129.29,Mozilla/5.0 (Linux; Android 6.0.1; SM-J510F Bu...,0,0,NaN,NaN,NaN,NaN,0
3721782,41ba908c-4621-4485-b0fe-46fb0d3a1892,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-25 10:26:19,NaN,NaN,0,...,uuid=41ba908c-4621-4485-b0fe-46fb0d3a1892&serv...,217.66.159.182,Mozilla/5.0 (Linux; Android 6.0.1; 5051D Build...,0,0,NaN,NaN,NaN,NaN,1
3721799,80f2d0e6-596f-4c1f-9168-d3f8e2025135,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-25 10:26:21,NaN,NaN,0,...,uuid=80f2d0e6-596f-4c1f-9168-d3f8e2025135&serv...,95.153.129.195,Mozilla/5.0 (Linux; Android 6.0.1; SM-G570F Bu...,0,0,NaN,NaN,NaN,NaN,0
3721899,8db852bc-94a5-49c9-866a-b0a666784c93,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-25 10:26:43,NaN,NaN,0,...,uuid=8db852bc-94a5-49c9-866a-b0a666784c93&serv...,80.83.239.81,Mozilla/5.0 (Linux; Android 6.0.1; SM-A500F Bu...,0,0,NaN,NaN,NaN,NaN,0
3721929,883993c7-cab1-418b-bce2-71c855537b1c,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-25 10:26:50,NaN,NaN,0,...,uuid=883993c7-cab1-418b-bce2-71c855537b1c&serv...,217.66.159.121,Mozilla/5.0 (Linux; Android 6.0; HUAWEI GRA-UL...,0,0,NaN,NaN,NaN,NaN,0
3721944,fe6bf351-1101-4a48-b365-c4c9182986bc,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-25 10:26:53,NaN,NaN,0,...,uuid=fe6bf351-1101-4a48-b365-c4c9182986bc&serv...,85.140.78.220,Mozilla/5.0 (Linux; Android 7.0; SM-G920F Buil...,0,0,NaN,NaN,NaN,NaN,0
3721970,9efe51f6-9029-461a-adfa-c34cfbab773f,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-25 10:27:01,NaN,NaN,0,...,uuid=9efe51f6-9029-461a-adfa-c34cfbab773f&serv...,85.140.77.239,Mozilla/5.0 (Linux; Android 6.0.1; SM-J510FN B...,0,0,NaN,NaN,NaN,NaN,0
3722023,8302461f-dbff-4bb0-b0fb-16911e178ece,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-25 10:27:14,NaN,NaN,0,...,uuid=8302461f-dbff-4bb0-b0fb-16911e178ece&serv...,213.87.224.9,Mozilla/5.0 (Linux; Android 5.0.2; SM-A300F Bu...,0,0,NaN,NaN,NaN,NaN,0
3722044,e2117e90-203d-47e5-80fe-e1574a2e14eb,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-25 10:27:17,NaN,NaN,0,...,uuid=e2117e90-203d-47e5-80fe-e1574a2e14eb&serv...,80.83.239.49,Mozilla/5.0 (Linux; Android 6.0.1; SM-J106F Bu...,0,0,NaN,NaN,NaN,NaN,0
3722081,b0d39b0c-5909-4a35-bb95-57d37b9df540,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-25 10:27:25,NaN,NaN,0,...,uuid=b0d39b0c-5909-4a35-bb95-57d37b9df540&serv...,213.87.249.53,Mozilla/5.0 (Linux; Android 7.0; E6653 Build/3...,0,0,NaN,NaN,NaN,NaN,0


In [41]:
next(warp_portal)

,sub_uuid,service_id,URL,Подключение,Юр.лицо,ОСС,date_init,date_activation,date_stop,msisdn,...,query,ip,ua,error_code,is_active,rebill_uuid,rebill_date,rebill_error_code,rebill_status,ua_black_list
130504,3cb1ffac-7845-11e7-93aa-638bdddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-03 15:14:02,NaN,NaN,0,...,utm_source=das&tmpl=amedia_def&exchange=mobild...,217.66.159.63,Mozilla/5.0 (Linux; Android 5.1.1; PSP5506DUO ...,0,0,NaN,NaN,NaN,NaN,0
130569,7ba06082-7845-11e7-ac68-648bdddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-03 15:15:48,NaN,NaN,0,...,utm_source=das&tmpl=amedia_def&exchange=mobild...,85.140.6.95,Mozilla/5.0 (Linux; Android 4.4.2; FS452 Build...,0,0,NaN,NaN,NaN,NaN,1
130587,8c2feb16-7845-11e7-b442-658bdddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-03 15:16:16,NaN,NaN,0,...,utm_source=das&tmpl=amedia_def&exchange=mobild...,213.87.133.155,Mozilla/5.0 (Linux; Android 6.0.1; HTC Desire ...,0,0,NaN,NaN,NaN,NaN,0
130592,8fc49948-7845-11e7-ac68-648bdddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-03 15:16:22,NaN,NaN,0,...,utm_source=das&tmpl=amedia_def&exchange=mobild...,217.66.152.24,Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5 Bui...,0,0,NaN,NaN,NaN,NaN,0
130602,9641a112-7845-11e7-93aa-638bdddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-03 15:16:32,NaN,NaN,0,...,utm_source=das&tmpl=amedia_def&exchange=mobild...,95.153.134.244,Mozilla/5.0 (Linux; Android 5.0.2; Lenovo S90-...,0,0,NaN,NaN,NaN,NaN,0
130603,95da15d8-7845-11e7-b442-658bdddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-03 15:16:32,NaN,NaN,0,...,utm_source=das&tmpl=amedia_def&exchange=mobild...,213.87.136.78,Mozilla/5.0 (Linux; Android 5.0.2; SM-G530H Bu...,0,0,NaN,NaN,NaN,NaN,0
130605,980da248-7845-11e7-ac68-648bdddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-03 15:16:35,NaN,NaN,0,...,utm_source=das&tmpl=amedia_def&exchange=mobild...,217.66.152.48,Mozilla/5.0 (Linux; Android 5.0; Lenovo A1000 ...,0,0,NaN,NaN,NaN,NaN,0
130607,97b87fde-7845-11e7-93aa-638bdddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-03 15:16:35,NaN,NaN,0,...,utm_source=das&tmpl=amedia_def&exchange=mobild...,95.153.128.233,Mozilla/5.0 (Linux; Android 4.4.4; SM-T561 Bui...,0,0,NaN,NaN,NaN,NaN,0
130609,9ac2731a-7845-11e7-899f-628bdddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-03 15:16:40,NaN,NaN,0,...,utm_source=das&tmpl=amedia_def&exchange=mobild...,95.153.131.4,Mozilla/5.0 (Linux; Android 5.1; IRBIS TZ90 Bu...,0,0,NaN,NaN,NaN,NaN,0
130610,9b71dad0-7845-11e7-ac68-648bdddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-03 15:16:41,NaN,NaN,0,...,utm_source=das&tmpl=amedia_def&exchange=mobild...,95.153.131.213,Mozilla/5.0 (Linux; Android 4.4.2; GT-I9190 Bu...,0,0,NaN,NaN,NaN,NaN,0


In [42]:
next(warp_portal)

,sub_uuid,service_id,URL,Подключение,Юр.лицо,ОСС,date_init,date_activation,date_stop,msisdn,...,query,ip,ua,error_code,is_active,rebill_uuid,rebill_date,rebill_error_code,rebill_status,ua_black_list
4925212,e807c6f0-8e4a-11e7-a0a0-d4b53b4b643c,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-31 15:50:05,NaN,NaN,0,...,utm_source=das&tmpl=amedia_got6&exchange=repeater,85.140.76.250,Mozilla/5.0 (Linux; Android 5.1; Benefit_S402 ...,14,0,NaN,NaN,NaN,NaN,1
4925224,e8d719f0-8e4a-11e7-a0a0-d4b53b4b643c,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-31 15:50:10,NaN,NaN,0,...,utm_source=das&tmpl=amedia_got6&exchange=repeater,85.140.1.24,Mozilla/5.0 (Linux; Android 6.0; 5041D Build/M...,14,0,NaN,NaN,NaN,NaN,1
4925227,ed98e04a-8e4a-11e7-9e1e-75ff3b4b643c,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-31 15:50:12,NaN,NaN,0,...,utm_source=das&tmpl=amedia_got6&exchange=repeater,85.140.76.250,Mozilla/5.0 (Linux; Android 5.1; Benefit_S402 ...,14,0,NaN,NaN,NaN,NaN,1
4925281,fc1e1f90-8e4a-11e7-aa67-6dff3b4b643c,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-31 15:50:36,NaN,NaN,0,...,utm_source=das&tmpl=amedia_got6&exchange=repeater,85.140.1.24,Mozilla/5.0 (Linux; Android 6.0; 5041D Build/M...,14,0,NaN,NaN,NaN,NaN,1
4925345,0bd3ea5a-8e4b-11e7-b910-72ff3b4b643c,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-31 15:51:03,NaN,NaN,0,...,utm_source=das&tmpl=amedia_got6&exchange=repeater,217.66.157.215,Mozilla/5.0 (Linux; Android 5.0.2; SAMSUNG SM-...,14,0,NaN,NaN,NaN,NaN,0
4925424,1f1955b4-8e4b-11e7-b910-72ff3b4b643c,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-31 15:51:35,NaN,NaN,0,...,utm_source=das&tmpl=amedia_got6&exchange=repeater,213.87.146.240,Mozilla/5.0 (Linux; Android 6.0.1; Redmi 4 Bui...,0,0,NaN,NaN,NaN,NaN,0
4925427,1f2ba1ec-8e4b-11e7-9e1e-75ff3b4b643c,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-31 15:51:35,NaN,NaN,0,...,utm_source=das&tmpl=amedia_got6&exchange=repeater,213.87.146.240,Mozilla/5.0 (Linux; Android 6.0.1; Redmi 4 Bui...,8,0,NaN,NaN,NaN,NaN,0
4925433,2178da5a-8e4b-11e7-9082-6fff3b4b643c,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-31 15:51:39,NaN,NaN,0,...,utm_source=das&tmpl=amedia_got6&exchange=repeater,85.140.1.24,Mozilla/5.0 (Linux; Android 6.0; 5041D Build/M...,14,0,NaN,NaN,NaN,NaN,1
4925481,2c5b4b9c-8e4b-11e7-8805-70ff3b4b643c,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-31 15:51:57,NaN,NaN,0,...,utm_source=das&tmpl=amedia_got6&exchange=repeater,85.140.1.24,Mozilla/5.0 (Linux; Android 6.0; 5041D Build/M...,14,0,NaN,NaN,NaN,NaN,1
4925595,44b5dc98-8e4b-11e7-9548-76ff3b4b643c,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-31 15:52:38,NaN,NaN,0,...,utm_source=das&tmpl=amedia_got6&exchange=repeater,85.140.4.205,Mozilla/5.0 (Linux; Android 6.0; LEAGOO M8 Bui...,8,0,NaN,NaN,NaN,NaN,0


In [43]:
next(warp_portal)

,sub_uuid,service_id,URL,Подключение,Юр.лицо,ОСС,date_init,date_activation,date_stop,msisdn,...,query,ip,ua,error_code,is_active,rebill_uuid,rebill_date,rebill_error_code,rebill_status,ua_black_list
4566971,7ea7e728-8cb3-11e7-90c5-96f23b4b643c,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-29 15:13:41,NaN,NaN,0,...,utm_source=das&tmpl=amedia_got6&exchange=repub...,78.107.65.18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,8,0,NaN,NaN,NaN,NaN,0
4842419,51e18d6c-8dd9-11e7-90cf-f5e93b4b643c,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-31 02:16:58,NaN,NaN,0,...,utm_source=das&tmpl=amedia_got6&exchange=repub...,80.83.239.51,Mozilla/5.0 (Linux; Android 6.0.1; Le X527 Bui...,14,0,NaN,NaN,NaN,NaN,0
4898514,db0f0ad4-8e2d-11e7-9570-52b53b4b643c,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-31 12:22:06,NaN,NaN,0,...,utm_source=das&tmpl=amedia_got6&exchange=repub...,80.83.237.5,Mozilla/5.0 (Linux; Android 5.1; Ixion ML250 B...,0,0,NaN,NaN,NaN,NaN,0


In [44]:
next(warp_portal)

,sub_uuid,service_id,URL,Подключение,Юр.лицо,ОСС,date_init,date_activation,date_stop,msisdn,...,query,ip,ua,error_code,is_active,rebill_uuid,rebill_date,rebill_error_code,rebill_status,ua_black_list
89158,5694736e-77c3-11e7-b08f-4b72dddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-02 23:44:11,NaN,NaN,0,...,utm_source=das&tmpl=amedia_def&exchange=sape&t...,78.107.65.18,Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build...,8,0,NaN,NaN,NaN,NaN,0
89543,49ba5a0e-77c4-11e7-b08f-4b72dddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-02 23:50:59,NaN,NaN,0,...,utm_source=das&tmpl=amedia_def&exchange=sape&t...,78.107.65.18,Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build...,8,0,NaN,NaN,NaN,NaN,0
90047,48873bd8-77c5-11e7-8f8d-4872dddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-02 23:58:06,NaN,NaN,0,...,utm_source=das&tmpl=amedia_def&exchange=sape&t...,78.107.65.18,Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build...,8,0,NaN,NaN,NaN,NaN,0
90271,000428ca-77c6-11e7-8f8d-4872dddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-03 00:03:14,NaN,NaN,0,...,utm_source=das&tmpl=amedia_def&exchange=sape&t...,78.107.65.18,Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build...,8,0,NaN,NaN,NaN,NaN,0
90346,4b0f9dfe-77c6-11e7-818a-4a72dddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-03 00:05:20,NaN,NaN,0,...,utm_source=das&tmpl=amedia_def&exchange=sape&t...,78.107.65.18,Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build...,8,0,NaN,NaN,NaN,NaN,0
130671,c5ca931c-7845-11e7-93aa-638bdddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-03 15:17:52,NaN,NaN,0,...,utm_source=das&tmpl=amedia_def&exchange=sape&t...,80.83.237.33,Mozilla/5.0 (Linux; U; Android 4.0.4; ru-ru; P...,0,0,NaN,NaN,NaN,NaN,0
132002,cc73ca4c-7847-11e7-ac68-648bdddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-03 15:32:22,NaN,NaN,0,...,utm_source=das&tmpl=amedia_def&exchange=sape&t...,85.140.5.239,Mozilla/5.0 (Linux; Android 4.4.2; Pure J Buil...,0,0,NaN,NaN,NaN,NaN,0
132406,dcbf2e86-7848-11e7-899f-628bdddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-03 15:39:59,NaN,NaN,0,...,utm_source=das&tmpl=amedia_def&exchange=sape&t...,85.140.2.42,Mozilla/5.0 (Linux; Android 4.1.1; C1605 Build...,0,0,NaN,NaN,NaN,NaN,0
133639,e5d7f8ec-784b-11e7-899f-628bdddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-03 16:01:43,NaN,NaN,0,...,utm_source=das&tmpl=amedia_def&exchange=sape&t...,217.66.152.55,Mozilla/5.0 (Linux; Android 4.4.2; 4009D Build...,0,0,NaN,NaN,NaN,NaN,1
135721,0ed34450-7850-11e7-b442-658bdddfe09a,933,Амедиатека,mks (478),МКС ООО,МТС,2017-08-03 16:31:30,NaN,NaN,0,...,utm_source=das&tmpl=amedia_def&exchange=sape&t...,217.66.157.25,Mozilla/5.0 (Linux; Android 4.1.1; MTC 970H Bu...,0,0,NaN,NaN,NaN,NaN,1


In [45]:
next(warp_portal)

,sub_uuid,service_id,URL,Подключение,Юр.лицо,ОСС,date_init,date_activation,date_stop,msisdn,...,query,ip,ua,error_code,is_active,rebill_uuid,rebill_date,rebill_error_code,rebill_status,ua_black_list


In [46]:
next(warp_portal)

,sub_uuid,service_id,URL,Подключение,Юр.лицо,ОСС,date_init,date_activation,date_stop,msisdn,...,query,ip,ua,error_code,is_active,rebill_uuid,rebill_date,rebill_error_code,rebill_status,ua_black_list


In [47]:
next(warp_portal)

,sub_uuid,service_id,URL,Подключение,Юр.лицо,ОСС,date_init,date_activation,date_stop,msisdn,...,query,ip,ua,error_code,is_active,rebill_uuid,rebill_date,rebill_error_code,rebill_status,ua_black_list


In [48]:
next(warp_portal)

,sub_uuid,service_id,URL,Подключение,Юр.лицо,ОСС,date_init,date_activation,date_stop,msisdn,...,query,ip,ua,error_code,is_active,rebill_uuid,rebill_date,rebill_error_code,rebill_status,ua_black_list


In [49]:
next(warp_portal)

,sub_uuid,service_id,URL,Подключение,Юр.лицо,ОСС,date_init,date_activation,date_stop,msisdn,...,query,ip,ua,error_code,is_active,rebill_uuid,rebill_date,rebill_error_code,rebill_status,ua_black_list


In [50]:
next(warp_portal)

,sub_uuid,service_id,URL,Подключение,Юр.лицо,ОСС,date_init,date_activation,date_stop,msisdn,...,query,ip,ua,error_code,is_active,rebill_uuid,rebill_date,rebill_error_code,rebill_status,ua_black_list


In [51]:
next(warp_portal)

,sub_uuid,service_id,URL,Подключение,Юр.лицо,ОСС,date_init,date_activation,date_stop,msisdn,...,query,ip,ua,error_code,is_active,rebill_uuid,rebill_date,rebill_error_code,rebill_status,ua_black_list


In [52]:
next(warp_portal)

,sub_uuid,service_id,URL,Подключение,Юр.лицо,ОСС,date_init,date_activation,date_stop,msisdn,...,query,ip,ua,error_code,is_active,rebill_uuid,rebill_date,rebill_error_code,rebill_status,ua_black_list


In [53]:
next(warp_portal)

,sub_uuid,service_id,URL,Подключение,Юр.лицо,ОСС,date_init,date_activation,date_stop,msisdn,...,query,ip,ua,error_code,is_active,rebill_uuid,rebill_date,rebill_error_code,rebill_status,ua_black_list


In [54]:
next(warp_portal)

,sub_uuid,service_id,URL,Подключение,Юр.лицо,ОСС,date_init,date_activation,date_stop,msisdn,...,query,ip,ua,error_code,is_active,rebill_uuid,rebill_date,rebill_error_code,rebill_status,ua_black_list


In [55]:
next(warp_portal)

,sub_uuid,service_id,URL,Подключение,Юр.лицо,ОСС,date_init,date_activation,date_stop,msisdn,...,query,ip,ua,error_code,is_active,rebill_uuid,rebill_date,rebill_error_code,rebill_status,ua_black_list


In [56]:
next(warp_portal)

,sub_uuid,service_id,URL,Подключение,Юр.лицо,ОСС,date_init,date_activation,date_stop,msisdn,...,query,ip,ua,error_code,is_active,rebill_uuid,rebill_date,rebill_error_code,rebill_status,ua_black_list


In [57]:
next(warp_portal)

StopIteration: 

In [ ]:
# осталось обернуть варп_порталы в варп_воронку и подумать что делать со статистической фукнцией.
# пока что это самая быстрая реализация, которая не упарываеться в оперативку и интерирует объекты.
# в принципе искомый результат уже достигнут - за один execute возвращает пачку датафреймоф, но....
# аааЙ! да работать дома над такими вещами заблесплатно вообще не прёт. но, конечно потом на работе сгодиться.

In [23]:
def stat(init, subs, stops, subs_active, subs_fr_error, subs_type_error, 
         rebills, rebills_status, rebills_fr_error, rebills_type_error,
         users, users_unic, users_unic_source, users_ip, users_unic_ip):
    init = init['date_init'].value_counts().sum()
    subs = subs['date_activation'].value_counts().sum()
    stops = stops['date_stop'].value_counts().sum()
    subs_active = subs_active['is_active'].value_counts()
    subs_fr_error = subs_fr_error['error_code'].value_counts(normalize=True)
    subs_type_error = subs_type_error['error_code'].value_counts()
    rebills = rebills['rebill_date'].value_counts().sum()
    rebills_status = rebills_status['rebill_status'].value_counts()
    rebills_fr_error = rebills_fr_error['rebill_error_code'].value_counts(normalize=True)
    rebills_type_error = rebills_type_error['rebill_error_code'].value_counts()
    users = users['msisdn'].value_counts().sum()
    users_unic = users_unic['msisdn'].drop_duplicates().values.tolist()
    users_unic_source = users_unic_source['referrer'].drop_duplicates().values.tolist()
    users_ip = users_ip['ip'].values.tolist()
    users_unic_ip = users_unic_ip['ip'].drop_duplicates().values.tolist()
    return print('Число инитов', init,'Число подписок', subs, 'Число стопов', stops,'\n','Состояние подписок','\n',
                 subs_active,'\n', 
                 'Частоты ошибок по подпискам','\n',subs_fr_error,'\n', 'Типы ошибок по подпискам','\n', subs_type_error, '\n',
                 'Число ребиллов', rebills, '\n','Успешность ребильности', '\n',rebills_status, '\n',
                 'Частоты ошибок по ребиллам', '\n',rebills_fr_error, '\n','Типы ошибок по ребиллам', '\n',rebills_type_error,'Число юзеров', users,'\n',
                 'Число уникальных юезров', len(users_unic),'\n', 'Число уникальных источников', len(users_unic_source),'\n',
                 'Число ip адресов', len(users_ip),'\n', 'Число уникальных ip адресов',len(users_unic_ip))